As noted in the main page, I found two different data sets on UFC matches on <a href="https://www.kaggle.com/">kaggle.com</a>.  The first starts in March of 2010 and goes until July of 2020.  The second starts with the very first match in November of 1993 and goes till June of 2019.

I did most of my analysis with the second data set because it has more fights.  I did use the first one for one specific report because it has a column the other does not have.

The data in the dataset had to be massaged and transformed a good deal to be more usable.  Some examples are

- I modified some column names to make them more user friendly.

- Each fighter in a UFC fight is placed into a "Red" or "Blue" category.  One analysis I tried was to predict whether Red or Blue won.  The winner's name is given, but I needed to add a column to say which color corner they were assigned to.

- Many of the columns in this dataset had to be split into multiple columns, because they came in the form of "13 of 16", meaning the fighter, for example, landed 13 punches out of 16 attempted punches.  "13 of 16" is difficule to use, so I wrote some functions to split that into several columns - the number attempted, the number succeeded, the ratio, etc.

- Some columns did not seem useful so I dropped them.  For example, referee and location.


I also did a few other things, like a column to show the name of the loser for easier visual analsys, and functions to show the record of a specific person.  These were not used in a machine learning algorithm, but are nice to have for analysis by a human.

I will walk through some of the ways the data must be changed and cleansed.  While working on this originally, these operations were not necessarily done in this order or at the beginning at all. Some were done later one when I realized the need.  For example, I did some exploratory analysis before running into trouble with NaN values.  But for the sake of this presentation, we will do most of the data cleaning up front as if we knew in advance we would need it.

One final note.  Here I use Jupyter to walk through the process and reasoning used for data preparation.  But I put all of this code into a class written using PyCharm, and other notebooks using this dataset import that class and call this code with just a few lines.  My view is that Jupyter is great for presentations and making graphs, but for most code that will be used repeatedly over the long term, I use an IDE.

Let's get started

In [1]:
# Import the usual suspects.
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

In [2]:
# Read the first data file
ufc = pd.read_csv('ufc-master.csv')

In [3]:
ufc.shape

(4292, 110)

In [4]:
# Read the primary data file
fights = pd.read_csv('raw_total_fight_data.csv', sep=';')

In [5]:
fights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5144 entries, 0 to 5143
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   R_fighter        5144 non-null   object
 1   B_fighter        5144 non-null   object
 2   R_KD             5144 non-null   int64 
 3   B_KD             5144 non-null   int64 
 4   R_SIG_STR.       5144 non-null   object
 5   B_SIG_STR.       5144 non-null   object
 6   R_SIG_STR_pct    5144 non-null   object
 7   B_SIG_STR_pct    5144 non-null   object
 8   R_TOTAL_STR.     5144 non-null   object
 9   B_TOTAL_STR.     5144 non-null   object
 10  R_TD             5144 non-null   object
 11  B_TD             5144 non-null   object
 12  R_TD_pct         5144 non-null   object
 13  B_TD_pct         5144 non-null   object
 14  R_SUB_ATT        5144 non-null   int64 
 15  B_SUB_ATT        5144 non-null   int64 
 16  R_PASS           5144 non-null   int64 
 17  B_PASS           5144 non-null   

In [6]:
fights.shape

(5144, 41)

In [7]:
fights.describe()

,R_KD,B_KD,R_SUB_ATT,B_SUB_ATT,R_PASS,B_PASS,R_REV,B_REV,last_round
count,5144.000000,5144.000000,5144.000000,5144.000000,5144.000000,5144.000000,5144.000000,5144.000000,5144.000000
mean,0.252333,0.180404,0.510886,0.361003,1.401439,0.831260,0.141719,0.134137,2.288297
std,0.523318,0.459646,0.948139,0.809795,2.301587,1.651515,0.427448,0.416798,1.003732
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.000000,3.000000
max,5.000000,4.000000,10.000000,7.000000,26.000000,14.000000,5.000000,3.000000,5.000000


In [8]:
# First, we need to do something about the NaN values
fights = fights.replace(np.NaN, 'None')

In [9]:
# Define a function to tell you who the loser is.

In [10]:
def find_loser(row):
    if row.Winner == row.R_fighter:
        return row.B_fighter
    elif row.Winner == row.B_fighter:
        return row.R_fighter
    else:
        return 'None'

In [11]:
fights['loser'] = fights.apply(find_loser, axis=1)

In [12]:
# Let's see an some rows.  We'll start by looking at Royce Gracie, an early UFC star.
fights[(fights.R_fighter == 'Royce Gracie') | (fights.B_fighter == 'Royce Gracie')][['Winner', 'win_by', 'loser']]

,Winner,win_by,loser
4590,Matt Hughes,KO/TKO,Royce Gracie
5099,None,Other,None
5107,Royce Gracie,Submission,Dan Severn
5109,Royce Gracie,Submission,Keith Hackney
5113,Royce Gracie,Submission,Ron van Clief
5117,Royce Gracie,Submission,Kimo Leopoldo
5121,Royce Gracie,KO/TKO,Patrick Smith
5122,Royce Gracie,Submission,Remco Pardoel
5124,Royce Gracie,Submission,Jason DeLucia
5128,Royce Gracie,Submission,Minoki Ichihara


In [13]:
# Now let's make a function to tell us if it was the person from the red corner or the blue corner who won.
# We will use this for question 2, "predicting" if red or blue won based on the stats of the fight

In [14]:
def winner_b_r(row):
    if row['Winner'] == row.R_fighter:
        return 'R'
    elif row['Winner'] == row.B_fighter:
        return 'B'
    else:
        return 'None'

In [15]:
fights['r_b_winner'] = fights.apply(lambda row: winner_b_r(row), axis=1)

In [16]:
fights.head()[['R_fighter', 'B_fighter', 'Winner', 'r_b_winner']]

,R_fighter,B_fighter,Winner,r_b_winner
0,Henry Cejudo,Marlon Moraes,Henry Cejudo,R
1,Valentina Shevchenko,Jessica Eye,Valentina Shevchenko,R
2,Tony Ferguson,Donald Cerrone,Tony Ferguson,R
3,Jimmie Rivera,Petr Yan,Petr Yan,B
4,Tai Tuivasa,Blagoy Ivanov,Blagoy Ivanov,B


In [17]:
fights[(fights.R_fighter == 'Royce Gracie') | (fights.B_fighter == 'Royce Gracie')][['Winner', 'win_by', 'loser', 'r_b_winner']]

,Winner,win_by,loser,r_b_winner
4590,Matt Hughes,KO/TKO,Royce Gracie,R
5099,None,Other,None,None
5107,Royce Gracie,Submission,Dan Severn,R
5109,Royce Gracie,Submission,Keith Hackney,R
5113,Royce Gracie,Submission,Ron van Clief,R
5117,Royce Gracie,Submission,Kimo Leopoldo,R
5121,Royce Gracie,KO/TKO,Patrick Smith,R
5122,Royce Gracie,Submission,Remco Pardoel,R
5124,Royce Gracie,Submission,Jason DeLucia,R
5128,Royce Gracie,Submission,Minoki Ichihara,R


I noticed a couple of things about column names.  One, most of the columsn start with a capital letter.  There is nothing wrong with that, but it is slightly easier to type when they are all lower case.  Another is that four columns ("R_SIG_STR.", "B_SIG_STR.", "R_TOTAL_STR.", and "B_TOTAL_STR.") end in a period.  Again, you can do an analysis that way, but it is easier to drop the ".".

You can do something like this

In [18]:
fights.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR.', 'B_SIG_STR.',
       'R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_PASS', 'B_PASS', 'R_REV', 'B_REV', 'R_HEAD', 'B_HEAD', 'R_BODY',
       'B_BODY', 'R_LEG', 'B_LEG', 'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH',
       'B_CLINCH', 'R_GROUND', 'B_GROUND', 'win_by', 'last_round',
       'last_round_time', 'Format', 'Referee', 'date', 'location',
       'Fight_type', 'Winner', 'loser', 'r_b_winner'],
      dtype='object')

In [19]:
def cleanse_column_names(fights):
    # First, deal with columns with a period at the end.
    cols_with_period = []
    new_column_names = {}
    for c in fights.columns:  # columns_of_interest:
        if c.endswith('.'):
            cols_with_period.append(c)
            new_column_names[c] = c.replace('.', '')
    fights = fights.rename(columns=new_column_names)

    # Now, make tham all lower case.
    col_names_map = {}
    for column in fights.columns:
        col_names_map[column] = column.lower()
    # col_names_map

    fights = fights.rename(columns=col_names_map)
    return fights

In [20]:
fights = cleanse_column_names(fights)

In [21]:
fights.columns

Index(['r_fighter', 'b_fighter', 'r_kd', 'b_kd', 'r_sig_str', 'b_sig_str',
       'r_sig_str_pct', 'b_sig_str_pct', 'r_total_str', 'b_total_str', 'r_td',
       'b_td', 'r_td_pct', 'b_td_pct', 'r_sub_att', 'b_sub_att', 'r_pass',
       'b_pass', 'r_rev', 'b_rev', 'r_head', 'b_head', 'r_body', 'b_body',
       'r_leg', 'b_leg', 'r_distance', 'b_distance', 'r_clinch', 'b_clinch',
       'r_ground', 'b_ground', 'win_by', 'last_round', 'last_round_time',
       'format', 'referee', 'date', 'location', 'fight_type', 'winner',
       'loser', 'r_b_winner'],
      dtype='object')

In [22]:
# We will also need to update the find_loser() and winner_b_r() functions.
def find_loser(row):
    if row.winner == row.r_fighter:
        return row.b_fighter
    elif row.winner == row.b_fighter:
        return row.r_fighter
    else:
        return 'None'

In [23]:
def winner_b_r(row):
    if row['winner'] == row.r_fighter:
        return 'R'
    elif row['winner'] == row.b_fighter:
        return 'B'
    else:
        return 'None'

In [24]:
fights['loser'] = fights.apply(find_loser, axis=1)
fights['r_b_winner'] = fights.apply(lambda row: winner_b_r(row), axis=1)

The next thing to deal with is all the columns that are of the form "16 of 27".  Let's look at the top row.  

In [25]:
fights.iloc[0]

r_fighter                         Henry Cejudo
b_fighter                        Marlon Moraes
r_kd                                         0
b_kd                                         0
r_sig_str                            90 of 171
b_sig_str                            57 of 119
r_sig_str_pct                              52%
b_sig_str_pct                              47%
r_total_str                          99 of 182
b_total_str                          59 of 121
r_td                                    1 of 4
b_td                                    0 of 2
r_td_pct                                   25%
b_td_pct                                    0%
r_sub_att                                    1
b_sub_att                                    0
r_pass                                       1
b_pass                                       0
r_rev                                        0
b_rev                                        0
r_head                               73 of 150
b_head       

You see things like R_SIG_STR = "90 of 171" and B_TOTAL_STR. = "59 OF 121"

This is more than aesthetic.  In order to run logistic regression or use a neural network, you want the columns you use to be numeric.
Let's look at r_sig_str, which stands for significant strikes by the Red fighter and has a value of "90 of 171".  From this, we know
-he attempted 171 significant strikes
-he landed 90
-he missed 81
-he landed approximately 52.6% of attempts

For the analysis, I guessed that the number missed would not be important, but the number of attempts, successes and ratio would be.  I also made a percent column, which is just ratio * 100.


In [26]:
# Some constants that will be used
ATTEMPT = '_att'
SUCCESS = '_suc'
PERCENT = '_pct'
RATIO = '_ratio'
DIFFERENCE = '_diff'
composite_column_stems = ['sig_str', 'total_str', 'td', 'head', 'body', 'leg', 'distance', 'clinch', 'ground']

In [27]:
def expand_column_names(column_root):
    r = 'r_'
    b = 'b_'
    att = ATTEMPT
    suc = SUCCESS
    ratio = RATIO
    pct = PERCENT
    diff = DIFFERENCE
    return [r + column_root + att, b + column_root + att, r + column_root + suc, b + column_root + suc,
            r + column_root + ratio, b + column_root + ratio, r + column_root + pct, b + column_root + pct,
            column_root + diff]

In [28]:
# example / test
expand_column_names('sig_str')

['r_sig_str_att',
 'b_sig_str_att',
 'r_sig_str_suc',
 'b_sig_str_suc',
 'r_sig_str_ratio',
 'b_sig_str_ratio',
 'r_sig_str_pct',
 'b_sig_str_pct',
 'sig_str_diff']

In [29]:
# Some functions will be use in the main column expansion function
def current_column_names(column_root):
    return ['r_' + column_root, 'b_' + column_root]

def split_column_text(text):
    string_values = text.split(' of ')
    return np.array([int(string_values[0]), int(string_values[1])])


In [30]:
current_column_names('sig_str')

['r_sig_str', 'b_sig_str']

In [31]:
split_column_text("90 of 171")

array([ 90, 171])

Now we can write the function to go through each column and split each out into the resulting numeric columns that the ML algorithms can use.

In [32]:
def split_composite_columns(df):
    for stem in composite_column_stems:
        expanded = expand_column_names(stem)
        current = current_column_names(stem)
        r_suc_att_values = df[current[0]].apply(lambda x: split_column_text(x))
        b_suc_att_values = df[current[1]].apply(lambda x: split_column_text(x))
        r_suc = r_suc_att_values.apply(lambda v: v[0])
        r_att = r_suc_att_values.apply(lambda v: v[1])
        b_suc = b_suc_att_values.apply(lambda v: v[0])
        b_att = b_suc_att_values.apply(lambda v: v[1])

        r_ratio_value = r_suc / r_att #r_suc_att_values[0] / r_suc_att_values[1]
        b_ratio_value = b_suc / b_att  # b_suc_att_values[0] / b_suc_att_values[1]

        # Now, add the new columns
        df[expanded[0]] = r_att
        df[expanded[1]] = b_att
        df[expanded[2]] = r_suc
        df[expanded[3]] = b_suc
        df[expanded[4]] = r_ratio_value.replace(np.NaN, 0.0)  # If it was "0 of 0", make the ratio 0, not np.NaN
        df[expanded[5]] = b_ratio_value.replace(np.NaN, 0.0)

        # Where in an existing _pct column, remove the '%' and make it an int, not a string.
        # When there is not an existing _pct column, create one.
        r_percent_col = f'r_{stem}{PERCENT}'
        if r_percent_col in df.columns:
            df[r_percent_col] = df[r_percent_col].str.replace('%', '').apply(lambda x: int(x))
        else:
            df[expanded[6]] = (100 * df[expanded[4]]).apply(lambda n: int(n))

        b_percent_col = f'b_{stem}{PERCENT}'
        if b_percent_col in df.columns:
            df[b_percent_col] = df[b_percent_col].str.replace('%', '').apply(lambda x: int(x))
        else:
            df[expanded[7]] = (100 * df[expanded[5]]).apply(lambda n: int(n))

        # TODO:  Is there a way of keeping track of columns that does not involve matching numbers?  This is
        #  annoying.
        df[expanded[8]] = r_suc - b_suc

        # Delete the old columns
        # df.drop(current[0], axis=1)
        # df.drop(current[1], axis=1)

    return df

In [33]:
# test of split_composite_columns

In [34]:
test_stems = ['sig_str', 'body', 'td'] # a scaled down list of columns to break out
composite_column_stems = test_stems
df = pd.DataFrame(data=
                          {'r_fighter': ['John Wayne', 'Ghenghis Khan', 'Billy the Kid', 'Aladdin'],
                           'b_fighter': ['Darth Vader', 'Severus Snape', 'Gandalf', 'Ali Babba'],
                           'r_sig_str': ['1 of 10', '3 of 15', '8 of 9', '0 of 2'],
                           'b_sig_str': ['4 of 5', '18 of 20', '6 of 7', '0 of 0'],
                           'r_sig_str_pct': ['10%', '20%', '89%', '0%'],
                           'b_sig_str_pct': ['80%', '90%', '86%', '0%'],
                           'r_body': ['1 of 5', '2 of 10', '4 of 12', '0 of 0'],
                           'r_body_pct': ['20%', '20%', '33%', '0%'],
                           'b_body': ['3 of 4', '9 of 10', '8 of 12', '0 of 1'],
                           'b_body_pct': ['75%', '90%', '67%', '0%'],
                           'r_td': ['0 of 0', '1 of 12', '3 of 4', '8 of 8'],
                           'b_td': ['20 of 20', '3 of 4', '9 of 9', '1 of 8']
                           })

In [35]:
result = split_composite_columns(df)

In [36]:
result

,r_fighter,b_fighter,r_sig_str,b_sig_str,r_sig_str_pct,b_sig_str_pct,r_body,r_body_pct,b_body,b_body_pct,...,body_diff,r_td_att,b_td_att,r_td_suc,b_td_suc,r_td_ratio,b_td_ratio,r_td_pct,b_td_pct,td_diff
0,John Wayne,Darth Vader,1 of 10,4 of 5,10,80,1 of 5,20,3 of 4,75,...,-2,0,20,0,20,0.000000,1.000,0,100,-20
1,Ghenghis Khan,Severus Snape,3 of 15,18 of 20,20,90,2 of 10,20,9 of 10,90,...,-7,12,4,1,3,0.083333,0.750,8,75,-2
2,Billy the Kid,Gandalf,8 of 9,6 of 7,89,86,4 of 12,33,8 of 12,67,...,-4,4,9,3,9,0.750000,1.000,75,100,-6
3,Aladdin,Ali Babba,0 of 2,0 of 0,0,0,0 of 0,0,0 of 1,0,...,0,8,8,8,1,1.000000,0.125,100,12,7


In [37]:
# Use the real stems list again.
composite_column_stems = ['sig_str', 'total_str', 'td', 'head', 'body', 'leg', 'distance', 'clinch', 'ground']

At this point, we are able to define some functions that can handle all of the above quickly, for every time we want to reload the dataset.

In [38]:
def cleanse(fights):
    """
    Calls the cleansing functions in one function call.
    :param fights:
    :return:
    """
    fights = cleanse_column_names(fights)
    fights = split_composite_columns(fights)
    fights = fights.replace(np.NaN, 'None')
    fights['loser'] = fights.apply(find_loser, axis=1)
    fights['r_b_winner'] = fights.apply(lambda row: winner_b_r(row), axis=1)
    return fights

In [39]:
def load_and_cleanse(filename, sep):
    """
    Combines loading the file and cleansing.
    :param filename:
    :param sep:
    :return:
    """
    fights = pd.read_csv(filename, sep=sep)
    fights = cleanse(fights)
    return fights

In [40]:
# Now, all we need to do is call this function to get a ready DataFrame

In [41]:
fights = load_and_cleanse('raw_total_fight_data.csv', sep=';')

In [42]:
fights

,r_fighter,b_fighter,r_kd,b_kd,r_sig_str,b_sig_str,r_sig_str_pct,b_sig_str_pct,r_total_str,b_total_str,...,b_ground_att,r_ground_suc,b_ground_suc,r_ground_ratio,b_ground_ratio,r_ground_pct,b_ground_pct,ground_diff,loser,r_b_winner
0,Henry Cejudo,Marlon Moraes,0,0,90 of 171,57 of 119,52,47,99 of 182,59 of 121,...,1,26,1,0.866667,1.000000,86,100,25,Marlon Moraes,R
1,Valentina Shevchenko,Jessica Eye,1,0,8 of 11,2 of 12,72,16,37 of 40,42 of 52,...,0,1,0,1.000000,0.000000,100,0,1,Jessica Eye,R
2,Tony Ferguson,Donald Cerrone,0,0,104 of 200,68 of 185,52,36,104 of 200,68 of 185,...,0,0,0,0.000000,0.000000,0,0,0,Donald Cerrone,R
3,Jimmie Rivera,Petr Yan,0,2,73 of 192,56 of 189,38,29,76 of 195,58 of 192,...,10,4,4,1.000000,0.400000,100,40,0,Jimmie Rivera,B
4,Tai Tuivasa,Blagoy Ivanov,0,1,64 of 144,73 of 123,44,59,66 of 146,81 of 131,...,6,0,6,0.000000,1.000000,0,100,-6,Tai Tuivasa,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5139,Gerard Gordeau,Kevin Rosier,1,0,11 of 17,0 of 3,64,0,11 of 17,0 of 3,...,0,6,0,0.666667,0.000000,66,0,6,Kevin Rosier,R
5140,Ken Shamrock,Patrick Smith,0,0,1 of 1,4 of 8,100,50,4 of 4,16 of 20,...,6,1,2,1.000000,0.333333,100,33,-1,Patrick Smith,R
5141,Royce Gracie,Art Jimmerson,0,0,0 of 3,0 of 0,0,0,4 of 7,0 of 0,...,0,0,0,0.000000,0.000000,0,0,0,Art Jimmerson,R
5142,Kevin Rosier,Zane Frazier,2,0,15 of 27,12 of 28,55,42,38 of 53,13 of 29,...,2,7,2,0.875000,1.000000,87,100,5,Zane Frazier,R


In [43]:
fights.shape

(5144, 120)

As mentioned earlier, I put these functions into a class, so other pages that use the code on this page will use it with just an import an a function call.